# Programowanie współbieżne i równoległe

## Wątki, zadania, synchronizacja, monitory, wartości atomowe, kolekcje

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@agh.edu.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe


Część przykładów pochodzi ze strony:
http://winterbe.com

# Java + Threads

<img src="img/thread.png" width="300"/>

# Program, proces, wątek

<img src="img/process_and_thread.png"/>

* program - plik lub zestaw plików opisujących w jakis sposób należy przetwarzać dane
* proces - uruchomiony program posiadający własną pamięć oraz licznik instrukcji
* wątek - lekki proces w obrębie działającego programu, posiadający własny stos oraz licznik instrukcji

# Klasa `Thread`

<img src="img/thread-1.png" />

API klasy `Thread`:
* `run()`
* `start()`
* `join()`
* `currentThread()`
* `getName()`
* ...

In [8]:
import static java.lang.System.out;
import java.util.List;
import java.util.LinkedList;

List<Thread> threads = new LinkedList<>();

out.println("Wątek główny " + Thread.currentThread().getName());

threads.add(new Thread(() -> out.println("Wątek " + Thread.currentThread().getName())));
threads.add(new Thread(() -> out.println("Wątek " + Thread.currentThread().getName())));
threads.add(new Thread(() -> out.println("Wątek " + Thread.currentThread().getName())));

threads.forEach(Thread::start);
for(Thread thread : threads){
    thread.join();
}

Wątek główny IJava-executor-0
Wątek Thread-22
Wątek Thread-24
Wątek Thread-23


# Interfejs `Runnable`

&nbsp;

<img src="img/runner.jpg" width="800"/>

API interfejsu `Runnable`:
* `run()`

In [43]:
import static java.lang.System.out;
import java.util.List;
import java.util.LinkedList;

List<Runnable> tasks = new LinkedList<>();

tasks.add(() -> out.println("Zadanie " + Thread.currentThread().getName()));
tasks.add(() -> out.println("Zadanie " + Thread.currentThread().getName()));
tasks.add(() -> out.println("Zadanie " + Thread.currentThread().getName()));

List<Thread> threads = new LinkedList<>();
tasks.forEach((task) -> threads.add(new Thread(task)));

threads.forEach(Thread::start);
//for(Thread t : threads) {
//        t.join();
//}


Zadanie Thread-127


# `Thread#sleep()`

&nbsp;
<img src="img/sleeping_cat.jpg"/>

In [47]:
import static java.lang.System.out;
import java.util.concurrent.*;

Thread sleepingThread = new Thread(() -> {
    try{
        out.println("Idę spać na 3 sekudny");
        TimeUnit.SECONDS.sleep(3);
        out.println("Godzinę później...");
    } catch (InterruptedException ex) {
        out.println("Sen został przerwany");
    }
});
sleepingThread.start();
sleepingThread.join();

Idę spać na 3 sekudny
Godzinę później...


# `ExecutorService`

&nbsp;

<img src="img/executor.jpg"/>

In [49]:
import java.util.concurrent.*;
import static java.lang.System.out;

ExecutorService executor =  Executors.newSingleThreadExecutor();
executor.submit(() -> out.println("Egzekucja w " + Thread.currentThread().getName()));
executor.submit(() -> out.println("Egzekucja w " + Thread.currentThread().getName()));
executor.submit(() -> out.println("Egzekucja w " + Thread.currentThread().getName()));
executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

Egzekucja w pool-2-thread-1
Egzekucja w pool-2-thread-1
Egzekucja w pool-2-thread-1


true

# `Future`

&nbsp;

<img src="img/future.jpg" width="800"/>

In [ ]:
interface Future<V> {
    boolean cancel(boolean mayInterruptIfRunning);
    V get();
    V get(long timeout, TimeUnit unit);
    boolean isCancelled();
    boolean isDone();
}

In [50]:
import java.util.concurrent.*;
final int sleepTime = 3;

Callable<Integer> task = () -> {
    try {
        TimeUnit.SECONDS.sleep(sleepTime);
        return 123;
    } catch (InterruptedException e) {
        throw new IllegalStateException("wątek został przerwany", e);
    }
};

ExecutorService executor = Executors.newFixedThreadPool(1);
Future<Integer> future = executor.submit(task);

out.println("obliczenie zakończone? " + future.isDone());

Integer result = future.get();

out.println("obliczenie zakończone? " + future.isDone());
out.print("wynik: " + result);

obliczenie zakończone? false
obliczenie zakończone? true
wynik: 123

# Rodzaje wykonawców (`ExecutorService`)

* `newCachedThreadPool` - tworzy wątki w zależności od potrzeb i usuwa je jeśli nie są używane przez 60 sekund
* `newFixedThreadPool` - cały czas przechowuje niezakończone wątki
* `newScheduledThreadPool` - posiada możliwość odroczonego i periodycznego wykonania wątków
* `newSingleThreadExecutor` - wykonanie jednowątkowe
* `newSingleThreadScheduledExecutor` - jw. ale z możliwością odroczonego i periodycznego wykonania


# `ScheduledExecutor`

In [53]:
import java.util.concurrent.*;
import static java.lang.System.out;
ScheduledExecutorService executor = Executors.newScheduledThreadPool(2);

Runnable task = () -> out.println("Wykonanie odroczonego zadania w " + Thread.currentThread().getName());
ScheduledFuture<?> future = executor.schedule(task, 3, TimeUnit.SECONDS);

out.println("Przed oczekiwaniem");

TimeUnit.MILLISECONDS.sleep(1000);

out.println("Czas pozostały do wykonania " + future.getDelay(TimeUnit.MILLISECONDS));
TimeUnit.SECONDS.sleep(3);
executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

Przed oczekiwaniem
Czas pozostały do wykonania 1930
Wykonanie odroczonego zadania w pool-6-thread-1


true

# Hazard (Race condition)

In [55]:
class RaceCondition {
    private int counter = 0;
    
    public void increment() {
        this.counter = this.counter + 1;
    }
    
    public int getCounter(){
        return this.counter;
    }
}

In [56]:
import java.util.concurrent.*;
import java.util.stream.*;

RaceCondition object = new RaceCondition();

IntStream.range(0, 1000000).forEach(i -> object.increment());

System.out.println(object.getCounter());

1000000


In [65]:
import java.util.concurrent.*;
import java.util.stream.*;

ExecutorService executor = Executors.newFixedThreadPool(2);

RaceCondition object = new RaceCondition();

IntStream.range(0, 1000000).forEach(i -> executor.submit(object::increment));

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

true

In [66]:
System.out.println(object.getCounter());

984990


<center><img src="img/race.jpg" width="400"/></center>

<center><img src="img/race.png" /></center>

# Sekcja krytyczna (`synchronized`)

In [67]:
class SynchronizedAccessors {
    private int counter = 0;

    public synchronized void increment() {
        this.counter = this.counter + 1;
    }

    public synchronized int getCounter(){
        return this.counter;
    }
}

In [68]:
import java.util.concurrent.*;
import java.util.stream.*;

ExecutorService executor = Executors.newFixedThreadPool(2);

SynchronizedAccessors object = new SynchronizedAccessors();

IntStream.range(0, 1000000).forEach(i -> executor.submit(object::increment));

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

System.out.println(object.getCounter());

1000000


In [69]:
class Monitor {
    private int counter = 0;

    public void increment() {
        synchronized(this) {
            this.counter += 1;
        }
    }

    public int getCounter(){
        synchronized(this) {
            return this.counter;
        }
    }
}

# `wait` i `notify`

In [71]:
import java.util.*;

class StringStack {
    private List<String> stack = new LinkedList<>();
    
    public void push(String value){
        synchronized(this) {
            stack.add(value);
            notify();
        }
    }
    
    public String pop(){
        synchronized(this) {
            while(stack.isEmpty()){
                try {
                    wait();
                } catch (InterruptedException ex) {
                    out.println("Wątek został przerwany");
                }
            }
            return stack.remove(stack.size() - 1);
        }
    }
}

In [75]:
StringStack stack = new StringStack();

Thread ideaProducer = new Thread(() -> {
    try {
        out.println("Filozof: Myślę");
        Thread.sleep(3000);
        out.println("Filozof: Produkuję myśl");
        stack.push("Myślę więc jestem");
        Thread.sleep(3000);
        out.println("Filozof: Produkuję myśl");
        stack.push("Różowe idee wściekle śpią");
    } catch (InterruptedException ex) {
        out.println("Wątek dodający został przerwany");
    }
});

Thread ideaConsumer = new Thread(() -> {
    out.println("Konsument: Czekam na jakąś mądrą myśl...");
    out.println("Konsument: Konsumuję myśl: " + stack.pop());
    out.println("Konsument: Konsumuję myśl: " + stack.pop());
});

ExecutorService executor = Executors.newFixedThreadPool(2);
executor.submit(ideaConsumer);
executor.submit(ideaProducer);
executor.shutdown();
executor.awaitTermination(7, TimeUnit.SECONDS);

Konsument: Czekam na jakąś mądrą myśl...
Filozof: Myślę
Filozof: Produkuję myśl
Konsument: Konsumuję myśl: Myślę więc jestem
Filozof: Produkuję myśl
Konsument: Konsumuję myśl: Różowe idee wściekle śpią


true

# Klasa `ReentrantLock`

In [ ]:
import java.util.concurrent.locks.*;

class ReentrantLockAccessors {
    private int counter = 0;
    private Lock lock = new ReentrantLock();

    public void increment() {
        lock.lock();
        try {
            this.counter += 1;
        } finally {
            lock.unlock();
        }
    }

    public int getCounter(){
        lock.lock();
        try {
            return this.counter;
        } finally {
            lock.unlock();
        }
    }
}

# `AtomicInteger`
&nbsp;

<center><img src="img/atomic.jpg" /></center>

In [1]:
import java.util.stream.*;
import java.util.concurrent.*;
import java.util.concurrent.atomic.*;

AtomicInteger atomicInt = new AtomicInteger(0);

ExecutorService executor = Executors.newFixedThreadPool(2);

IntStream.range(0, 1000000).forEach(i -> executor.submit(atomicInt::incrementAndGet));

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);
System.out.println(atomicInt.get());

1000000


In [2]:
AtomicInteger atomicInt = new AtomicInteger(0);

ExecutorService executor = Executors.newFixedThreadPool(2);

IntStream.range(0, 10000).forEach(i -> {
    executor.submit(() -> atomicInt.updateAndGet(n -> n + i));
});

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

System.out.println(atomicInt.get());

49995000


# `AtomictInteger`

* `addAndGet`
* `compareAndSet`
* `decrementAndGet`
* `get`
* `getAndAdd`
* `getAndDecrement`
* `getAndIncrement`
* `getAndSet`
* ...

# `LongAdder`

In [3]:
import java.util.stream.*;
import java.util.concurrent.*;
import java.util.concurrent.atomic.*;

LongAdder adder = new LongAdder();
ExecutorService executor = Executors.newFixedThreadPool(4);

IntStream.range(0, 1000000).forEach(i -> executor.submit(adder::increment));

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

System.out.println(adder.sumThenReset()); 

1000000


<center><img src="img/long_adder.png" width="800"/></center>

# `LongAccumulator`

In [5]:
import java.util.stream.*;
import java.util.concurrent.*;
import java.util.concurrent.atomic.*;
import java.util.function.*;

LongBinaryOperator operation = (x, y) -> x + y;
LongAccumulator accumulator = new LongAccumulator(operation, 0);

ExecutorService executor = Executors.newFixedThreadPool(4);

IntStream.range(0, 1000000).forEach(i -> executor.submit(() -> accumulator.accumulate(i)));

executor.shutdown();
executor.awaitTermination(1, TimeUnit.SECONDS);

System.out.println(accumulator.getThenReset());

499999500000


Operacja musi być przemienna, w przeciwnym razie wynik będzie niepoprawny.

# Przetwarzanie współbieżne a kolekcje

* `CopyOnWriteArrayList`
* `ConcurrentHashMap`
* `parallelStream`

# ConcurrentModificationException

In [6]:
class ParallelLinkedList {
    public void run() throws InterruptedException {
        List<Integer> list = new LinkedList<>();  // <-----------------------
        ExecutorService executor = Executors.newFixedThreadPool(200);
        IntStream.range(1,50).forEach((i) -> executor.submit(() -> { 
              System.out.println("" + i + " writing to LinkedList " + Thread.currentThread().getName());
              list.add(i); 
        }));
        executor.submit(() -> {
              Iterator<Integer> iterator = list.iterator();
              try {
                  while(iterator.hasNext()){  
                          Thread.sleep(20);
                          System.out.println("" + iterator.next() + " reading from LinkedList " + 
                              Thread.currentThread().getName());
                  }
              } catch(Exception ex) {
                  ex.printStackTrace();
              }
        });
        IntStream.range(1,50).forEach((i) -> executor.submit(() -> { 
              System.out.println("" + i + " writing to LinkedList " + Thread.currentThread().getName());
              list.add(i);
        }));
        executor.shutdown();
        executor.awaitTermination(1, TimeUnit.SECONDS);
    }
}

In [7]:
new ParallelLinkedList().run();

2 writing to LinkedList pool-6-thread-2
3 writing to LinkedList pool-6-thread-3
1 writing to LinkedList pool-6-thread-1
4 writing to LinkedList pool-6-thread-4
5 writing to LinkedList pool-6-thread-5
6 writing to LinkedList pool-6-thread-6
7 writing to LinkedList pool-6-thread-7
11 writing to LinkedList pool-6-thread-11
13 writing to LinkedList pool-6-thread-13
16 writing to LinkedList pool-6-thread-16
10 writing to LinkedList pool-6-thread-10
25 writing to LinkedList pool-6-thread-25
27 writing to LinkedList pool-6-thread-27
9 writing to LinkedList pool-6-thread-9
8 writing to LinkedList pool-6-thread-8
30 writing to LinkedList pool-6-thread-30
29 writing to LinkedList pool-6-thread-29
28 writing to LinkedList pool-6-thread-28
26 writing to LinkedList pool-6-thread-26
24 writing to LinkedList pool-6-thread-24
23 writing to LinkedList pool-6-thread-23
22 writing to LinkedList pool-6-thread-22
21 writing to LinkedList pool-6-thread-21
20 writing to LinkedList pool-6-thread-20
19 writing

java.util.ConcurrentModificationException
	at java.base/java.util.LinkedList$ListItr.checkForComodification(LinkedList.java:970)
	at java.base/java.util.LinkedList$ListItr.next(LinkedList.java:892)
	at REPL.$JShell$40$ParallelLinkedList.lambda$run$2($JShell$40.java:30)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:539)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:635)
	at java.base/java.lang.Thread.run(Thread.java:833)


15 writing to LinkedList pool-6-thread-65
14 writing to LinkedList pool-6-thread-64
13 writing to LinkedList pool-6-thread-63
12 writing to LinkedList pool-6-thread-62
17 writing to LinkedList pool-6-thread-67
18 writing to LinkedList pool-6-thread-68
19 writing to LinkedList pool-6-thread-69
20 writing to LinkedList pool-6-thread-70
21 writing to LinkedList pool-6-thread-71
22 writing to LinkedList pool-6-thread-72
23 writing to LinkedList pool-6-thread-73
24 writing to LinkedList pool-6-thread-74
25 writing to LinkedList pool-6-thread-75
26 writing to LinkedList pool-6-thread-76
27 writing to LinkedList pool-6-thread-77
28 writing to LinkedList pool-6-thread-78
29 writing to LinkedList pool-6-thread-79
30 writing to LinkedList pool-6-thread-80
33 writing to LinkedList pool-6-thread-83
32 writing to LinkedList pool-6-thread-82
31 writing to LinkedList pool-6-thread-81
36 writing to LinkedList pool-6-thread-86
35 writing to LinkedList pool-6-thread-85
34 writing to LinkedList pool-6-th

# `CopyOnWriteArrayList`

In [9]:
import java.util.*;
import java.util.stream.*;
import java.util.concurrent.*;
import java.util.concurrent.atomic.*;

class ParallelCopyOnWriteArrayList {
    public void run() throws InterruptedException {
        List<Integer> list = new CopyOnWriteArrayList<>();  // <-----------------------
        ExecutorService executor = Executors.newFixedThreadPool(200);
        IntStream.range(1,50).forEach((i) -> executor.submit(() -> { 
              //System.out.println("" + i + " writing to CopyOnWriteArrayList " + Thread.currentThread().getName());
              list.add(i); 
        }));
        executor.submit(() -> {
              Iterator<Integer> iterator1 = list.iterator();
              while(iterator1.hasNext()){
                    try {
                        Thread.sleep(20);
                        System.out.println("" + iterator1.next() + " reading from CopyOnWriteArrayList " + 
                        Thread.currentThread().getName());
                    } catch(Exception ex) {
                        ex.printStackTrace();
                    }
              }
        });
        IntStream.range(1,50).forEach((i) -> executor.submit(() -> { 
              //System.out.println("" + i + " writing to CopyOnWriteArrayList " + Thread.currentThread().getName());
              list.add(i); 
        }));
        executor.shutdown();
        executor.awaitTermination(1, TimeUnit.SECONDS);
    }
}

In [11]:
new ParallelCopyOnWriteArrayList().run();

1 reading from CopyOnWriteArrayList pool-8-thread-50
2 reading from CopyOnWriteArrayList pool-8-thread-50
3 reading from CopyOnWriteArrayList pool-8-thread-50
4 reading from CopyOnWriteArrayList pool-8-thread-50
5 reading from CopyOnWriteArrayList pool-8-thread-50
6 reading from CopyOnWriteArrayList pool-8-thread-50
7 reading from CopyOnWriteArrayList pool-8-thread-50
8 reading from CopyOnWriteArrayList pool-8-thread-50
9 reading from CopyOnWriteArrayList pool-8-thread-50
10 reading from CopyOnWriteArrayList pool-8-thread-50
11 reading from CopyOnWriteArrayList pool-8-thread-50
12 reading from CopyOnWriteArrayList pool-8-thread-50
13 reading from CopyOnWriteArrayList pool-8-thread-50
14 reading from CopyOnWriteArrayList pool-8-thread-50
15 reading from CopyOnWriteArrayList pool-8-thread-50
16 reading from CopyOnWriteArrayList pool-8-thread-50
17 reading from CopyOnWriteArrayList pool-8-thread-50
18 reading from CopyOnWriteArrayList pool-8-thread-50
19 reading from CopyOnWriteArrayList 

# `CommonPoolParallelism`

In [12]:
import java.util.concurrent.*;

System.out.println(ForkJoinPool.getCommonPoolParallelism());

//-Djava.util.concurrent.ForkJoinPool.common.parallelism=5

7


# `ConcurrentHashMap`

In [13]:
import java.util.concurrent.*;

ConcurrentHashMap<String, String> map = new ConcurrentHashMap<>();
map.put("1", "jeden");
map.put("2", "dwa");
map.put("3", "trzy");
map.put("4", "cztery");
map.put("5", "pięć");
map.put("6", "sześć");
map.put("7", "siedem");

map.forEach(1, (key, value) -> System.out.printf("klucz: %s; wartość: %s; wątek: %s\n",
        key, value, Thread.currentThread().getName()));

klucz: 1; wartość: jeden; wątek: IJava-executor-7
klucz: 2; wartość: dwa; wątek: ForkJoinPool.commonPool-worker-3
klucz: 3; wartość: trzy; wątek: ForkJoinPool.commonPool-worker-3
klucz: 4; wartość: cztery; wątek: ForkJoinPool.commonPool-worker-2
klucz: 6; wartość: sześć; wątek: IJava-executor-7
klucz: 5; wartość: pięć; wątek: ForkJoinPool.commonPool-worker-2
klucz: 7; wartość: siedem; wątek: IJava-executor-7


# `search`

In [16]:
String result = map.search(1, (key, value) -> {
    System.out.println(Thread.currentThread().getName());
    if ("dwa".equals(value)) {
        return key;
    }
    return null;
});
System.out.println("Wynik: " + result);

IJava-executor-8
ForkJoinPool.commonPool-worker-8
ForkJoinPool.commonPool-worker-8
ForkJoinPool.commonPool-worker-4
ForkJoinPool.commonPool-worker-4
ForkJoinPool.commonPool-worker-5
Wynik: 2


# `reduce`

In [19]:
String result = map.reduce(1,
    (key, value) -> {
        //System.out.println("Przekształcenie: " + Thread.currentThread().getName());
        return key + "=" + value;
    },
    (s1, s2) -> {
        //System.out.println("Redukcja: " + Thread.currentThread().getName());
        return s1 + ", " + s2;
    });

System.out.println("Wynik: " + result);

Wynik: 1=jeden, 2=dwa, 3=trzy, 4=cztery, 5=pięć, 6=sześć, 7=siedem


# `stream`

In [20]:
import java.util.stream.*;


long start = System.nanoTime();
long multiplier = 1000;

double result = LongStream.range(0, 1000000 * multiplier).filter(i -> i * i % 7 != 0).
    average().getAsDouble();

long end = System.nanoTime();

System.out.println(result);
System.out.println((end - start) * 1.0 / 1000000 * multiplier);

4.999999999166667E8
2864921.409


# `parallel` oraz `parallelStream`

In [21]:
import java.util.stream.*;

long start = System.nanoTime();
long multiplier = 1000;

double result = LongStream.range(0, 1000000 * multiplier).parallel().filter(i -> i * i % 7 != 0).
    average().getAsDouble();

long end = System.nanoTime();

System.out.println(result);
System.out.println((end - start) * 1.0 / 1000000 * multiplier);

4.999999999166667E8
927851.2050000001


Uwaga: podobnie jak w przypadku metod na `ConcurrentHashMap`, metoda `parallel` tworzy strumień korzystający z CommonPool, co może mieć negatywne konsekwencje, jeśli jakiś wątek w tej puli będzie przetwarzał się bardzo długo. https://www.baeldung.com/java-8-parallel-streams-custom-threadpool

# Współbieżność w JavaFX

* `Platform.runLater(Runnable r)`
* `javafx.concurrent.Task`
* `javafx.concurrent.Service`

# runLater

In [ ]:
// źródło: http://tutorials.jenkov.com/javafx/concurrency.html

public class MyApplication extends Application {
  public static void main(String[] args) {
    launch(args);
  }

  @Override
  public void start(Stage primaryStage) {
    primaryStage.setTitle("JavaFX App");

    ProgressBar progressBar = new ProgressBar(0);

    VBox vBox = new VBox(progressBar);
    Scene scene = new Scene(vBox, 960, 600);

    primaryStage.setScene(scene);
    primaryStage.show();

    Thread taskThread = new Thread(new Runnable() {
      @Override
      public void run() {
        double progress = 0;
        for(int i=0; i<10; i++){
          try {
            Thread.sleep(1000);
          } catch (InterruptedException e) {
            e.printStackTrace();
          }

          progress += 0.1;
          final double reportedProgress = progress;

          Platform.runLater(new Runnable() {              // <-----
            @Override
            public void run() {
              progressBar.setProgress(reportedProgress);
            }
          });
        }
      }
    });

    taskThread.start();
  }
}

# Task

In [ ]:
// źródło: https://docs.oracle.com/javafx/2/threads/jfxpub-threads.htm

import javafx.concurrent.Task;

Task task = new Task<Void>() {
    @Override public Void call() {
        static final int max = 1000000;
        for (int i=1; i<=max; i++) {
            if (isCancelled()) {
               break;
            }
            updateProgress(i, max);
        }
        return null;
    }
};
ProgressBar bar = new ProgressBar();
bar.progressProperty().bind(task.progressProperty());
new Thread(task).start();

# Service

In [ ]:
// źródło: https://docs.oracle.com/javafx/2/threads/jfxpub-threads.htm

public class FirstLineServiceApp extends Application {

    @Override
    public void start(Stage stage) throws Exception {
        FirstLineService service = new FirstLineService();
        service.setUrl("http://google.com");
        service.setOnSucceeded(new EventHandler<WorkerStateEvent>() {
            @Override
            public void handle(WorkerStateEvent t) {
                System.out.println("done:" + t.getSource().getValue());
            }
        });
        service.start();
    }

    public static void main(String[] args) {
        launch();
    }
}

In [ ]:
class FirstLineService extends Service<String> {
    private StringProperty url = new SimpleStringProperty();

    public final void setUrl(String value) {
        url.set(value);
    }

    public final String getUrl() {
        return url.get();
    }

    public final StringProperty urlProperty() {
       return url;
    }

    protected Task<String> createTask() {
        final String _url = getUrl();
        return new Task<String>() {
            protected String call() 
                throws IOException, MalformedURLException {
                    String result = null;
                    BufferedReader input = null;
                    try {
                        URL currentUrl = new URL(_url);
                        input = new BufferedReader(new InputStreamReader(currentUrl.openStream()));
                        result = input.readLine();
                    } finally {
                        if (input != null) {
                            input.close();
                        }
                    }
                    return result;
            }
        };
    }
}

![Pytania? ](img/question.jpg)